## 2022-10-2122 SN 2022vqz

*An analysis pipeline for our second night of supernova observation*

For additional pipeline ideas, we may want to study [a notebook by Vivek Kumar Jha](https://github.com/viveikjha/aperture_photometry/blob/master/Reduction_demonstration.ipynb) from Aryabhatta Research Institute of Observational Sciences ([ARIES](https://www.aries.res.in)) and [a notebook by Erik Tollerud](https://notebook.community/LSSTC-DSFP/LSSTC-DSFP-Sessions/Session5/Day1/python_imred) from the Space Telescope Science Institute ([STScI](https://www.stsci.edu)).

### Calibrate

We are using a slightly non-standard but pleasantly clean calibration procedure which requires
the lights and the darks to have the same exposure duration (typically of order 30s),
and the flats and the biases to have the same exposure duration (typically 1s). The more standard but less clean
procedure involves "scaling" the darks.

In [1]:
import os
import numpy as np
from astropy import units as u
from astropy.nddata import CCDData
from astropy.io import fits
from ccdproc import ImageFileCollection, Combiner, combine, subtract_dark, flat_correct
from astropy.visualization import imshow_norm, PercentileInterval, MinMaxInterval, SqrtStretch, LogStretch
import matplotlib.pyplot as plt
%matplotlib inline
import astroalign as aa
from photutils.aperture import RectangularAperture, RectangularAnnulus

ModuleNotFoundError: No module named 'photutils'

In [ ]:
# for an analyses of an observation run done with only a single filter,
# only the observation_date and the exposure times
# will need to be changed

observation_date = '2022-10-2425'
light_exposure = 30 * u.second
dark_exposure = light_exposure  # our method presumes this equality
flat_exposure = 1 * u.second
bias_exposure = flat_exposure  # our method presumes this equality

In [ ]:
# directory where the data being analyzed is kept

data_directory = os.path.join(os.path.expanduser('~'), 'Transients', observation_date)

In [ ]:
# subdirectories -- all relative to data_directory

light_directory = os.path.join(data_directory, 'light')
dark_directory = os.path.join(data_directory, 'dark')
flat_directory = os.path.join(data_directory, 'flat')
bias_directory = os.path.join(data_directory, 'bias')

# the aligned directory is *written to* not read from

aligned_directory = os.path.join(data_directory, 'aligned')
if not os.path.exists(aligned_directory):
    os.makedirs(aligned_directory)

In [ ]:
light_files = ImageFileCollection(light_directory).files_filtered(include_path='True')
dark_files = ImageFileCollection(dark_directory).files_filtered(include_path='True')
flat_files = ImageFileCollection(flat_directory).files_filtered(include_path='True')
bias_files = ImageFileCollection(bias_directory).files_filtered(include_path='True')

In [ ]:
lights = [CCDData.read(file, unit=u.adu) for file in light_files]
darks = [CCDData.read(file, unit=u.adu) for file in dark_files]
flats = [CCDData.read(file, unit=u.adu) for file in flat_files]
biases = [CCDData.read(file, unit=u.adu) for file in bias_files]

In [ ]:
method = 'median'  # the method can be changed to 'average'

master_dark = combine(darks, method=method)
master_flat = combine(flats, method=method)
master_bias = combine(biases, method=method)

In [ ]:
lights_subtracted = [subtract_dark(light,
                                   master_dark,
                                   data_exposure=light_exposure,
                                   dark_exposure=dark_exposure,
                                   scale=False)
                     for light in lights]

master_flat_subtracted = subtract_dark(master_flat,
                                       master_bias,
                                       data_exposure=flat_exposure,
                                       dark_exposure=bias_exposure,
                                       scale=False)

In [ ]:
lights_calibrated = [flat_correct(l, master_flat_subtracted) for l in lights_subtracted]

### Examine Calibration

Here we experiment with LogStretch and SqrtStretch from astropy.visualization. Principally, we follow the examples in the [Image stretching and normalization](https://docs.astropy.org/en/stable/visualization/normalization.html) documentation from astropy.visualization.

In [ ]:
light = lights[0]

In [ ]:
calibrated_light = lights_calibrated[0]

In [ ]:
interval_95 = PercentileInterval(95)
sqrt_stretch = SqrtStretch()
log_stretch = LogStretch()

fig, axes = plt.subplots(1, 2, figsize=(10, 10))

im_l, norm_l = imshow_norm(light, axes[0], cmap='gray', origin='lower',
                       interval=interval_95, stretch=sqrt_stretch)
axes[0].set_title("Light")

im_c_l, norm_c_l = imshow_norm(calibrated_light, axes[1], cmap='gray', origin='lower',
                       interval=interval_95, stretch=sqrt_stretch)
axes[1].set_title("Calibrated Light")

plt.tight_layout()
plt.show()

Our calibrated images appear to have both vignetting and the shadows of dust motes well-removed.

### Create and Write the Aligned Lights

From here on, we only have use for the calibrated lights. The last step before combining them is alignment.

In [ ]:
# This attempt to rotate and zoom lights_calibrated[1] is not working.
# The idea was to stress-test aa.register().
# Just comment the stress test out for now.

# from scipy.ndimage import rotate, zoom
# image_to_rotate = lights_calibrated[1]
# image_rotated = rotate(image_to_rotate, angle=30.0, reshape=False)
# image_rotated_and_zoomed = zoom(image_rotated, 1.5, order=2)
# lights_calibrated[1] = image_rotated_and_zoomed

In [ ]:
reference_light = lights_calibrated[0]

In [ ]:
lights_aligned_with_footprints = [aa.register(l, reference_light, detection_sigma=3.0) for l in lights_calibrated]

In [ ]:
for i in range(0, len(lights)):
    light_header = lights[i][0].header
    light_aligned_data = lights_aligned_with_footprints[i][0]
    aligned_file = os.path.join(aligned_directory, os.path.basename(light_files[i]))
    aligned_file2 = os.path.splitext(aligned_file)[0] + '_aligned.fit'
    fits.writeto(aligned_file2, light_aligned_data, light_header, overwrite=True)

### Examine Alignment

Let's examine the last aligned image in comparison with the first (reference) image.

In [ ]:
light = lights_calibrated[-1]

In [ ]:
light_aligned, footprint = lights_aligned_with_footprints[-1]

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

_, _ = imshow_norm(light, axes[0, 0], cmap='gray', origin='lower',
           interval=interval_95, stretch=sqrt_stretch)
axes[0, 0].set_title("Light")

_, _ = imshow_norm(reference_light, axes[0, 1], cmap='gray', origin='lower',
           interval=interval_95, stretch=sqrt_stretch)
axes[0, 1].set_title("Reference Light")

_, _ = imshow_norm(light_aligned, axes[1, 0], cmap='gray', origin='lower',
           interval=interval_95, stretch=sqrt_stretch)
axes[1, 0].set_title("Light Aligned")

axes[1, 1].imshow(footprint, cmap='gray', interpolation='none', origin='lower')
axes[1, 1].set_title("Footprint")

plt.tight_layout()
plt.show()

The auto-guider locked on so well that there is nothing to see.

### Combine the Aligned Lights

In [ ]:
aligned_files = ImageFileCollection(aligned_directory).files_filtered(include_path='True')
combined_light = combine([CCDData.read(file, unit=u.adu) for file in aligned_files], method=method)

### Examine the Combined Light

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(10, 10))

# log_stretch = LogStretch()  # the default log stretch is not as good as the default square-root stretch

_, _ = imshow_norm(combined_light, axes, cmap='gray', origin='lower',
                   interval=interval_95, stretch=sqrt_stretch)
axes.set_title("Combined Light")

plt.tight_layout()
plt.show()

### Perform Aperture-Annulus Photometry

In [6]:
positions = [(30., 30.), (40., 40.)]
aperture_width, aperture_height = 10.0, 10.0
annulus_inner_width, annulus_inner_height = 20.0, 20.0
annulus_outer_width, annulus_outer_height = 20.0, 20.0

In [7]:
RectangularAperture[positions, aperture_width, aperture_height]

NameError: name 'RectangularAperture' is not defined